In [52]:
import requests
import json
url = "https://api-basketball.p.rapidapi.com/"
headers = {
    "X-RapidAPI-Host": "api-basketball.p.rapidapi.com",
    "X-RapidAPI-Key": "api-key-here"
}

In [26]:
#NBA id = 12


{
    "errors": [],
    "get": "games",
    "parameters": {
        "h2h": "132-134"
    },
    "response": [
        {
            "country": {
                "code": "US",
                "flag": "https://media.api-sports.io/flags/us.svg",
                "id": 5,
                "name": "USA"
            },
            "date": "2015-11-05T01:00:00+00:00",
            "id": 26090,
            "league": {
                "id": 12,
                "logo": "https://media.api-sports.io/basketball/leagues/12.png",
                "name": "NBA",
                "season": "2015-2016",
                "type": "League"
            },
            "scores": {
                "away": {
                    "over_time": null,
                    "quarter_1": 17,
                    "quarter_2": 26,
                    "quarter_3": 27,
                    "quarter_4": 17,
                    "total": 87
                },
                "home": {
                    "over_time": null,
           

In [87]:
params = 'standings?league=12&season=2020-2021'
response = requests.get(url + params, headers=headers).json()
print(json.dumps(response, indent=4))

{
    "get": "standings",
    "parameters": {
        "league": "12",
        "season": "2020-2021"
    },
    "errors": [],
    "results": 1,
    "response": [
        [
            {
                "position": 1,
                "stage": "NBA - Regular Season",
                "group": {
                    "name": "Western Conference",
                    "points": null
                },
                "team": {
                    "id": 160,
                    "name": "Utah Jazz",
                    "logo": "https://media.api-sports.io/basketball/teams/160.png"
                },
                "league": {
                    "id": 12,
                    "name": "NBA",
                    "type": "League",
                    "season": "2020-2021",
                    "logo": "https://media.api-sports.io/basketball/leagues/12.png"
                },
                "country": {
                    "id": 5,
                    "name": "USA",
                    "code": "US",


In [88]:
weststandings = []
eaststandings = []
for team in response["response"][0]:
    if team["group"]["name"] == "Western Conference":
        weststandings.append((team["team"]["id"], team["team"]["name"], team["games"]["win"]["total"], team["games"]["lose"]["total"]))
    if team["group"]["name"] == "Eastern Conference":
        eaststandings.append((team["team"]["id"], team["team"]["name"], team["games"]["win"]["total"], team["games"]["lose"]["total"]))


        

In [89]:
weststandings

[(160, 'Utah Jazz', 52, 20),
 (155, 'Phoenix Suns', 51, 21),
 (139, 'Denver Nuggets', 47, 25),
 (144, 'Los Angeles Clippers', 47, 25),
 (138, 'Dallas Mavericks', 42, 30),
 (156, 'Portland Trail Blazers', 42, 30),
 (145, 'Los Angeles Lakers', 42, 30),
 (141, 'Golden State Warriors', 39, 33),
 (146, 'Memphis Grizzlies', 38, 34),
 (158, 'San Antonio Spurs', 33, 39),
 (150, 'New Orleans Pelicans', 31, 41),
 (157, 'Sacramento Kings', 31, 41),
 (149, 'Minnesota Timberwolves', 23, 49),
 (152, 'Oklahoma City Thunder', 22, 50),
 (142, 'Houston Rockets', 17, 55)]

In [90]:
eaststandings

[(154, 'Philadelphia 76ers', 49, 23),
 (134, 'Brooklyn Nets', 48, 24),
 (148, 'Milwaukee Bucks', 46, 26),
 (151, 'New York Knicks', 41, 31),
 (132, 'Atlanta Hawks', 41, 31),
 (147, 'Miami Heat', 40, 32),
 (133, 'Boston Celtics', 36, 36),
 (161, 'Washington Wizards', 34, 38),
 (143, 'Indiana Pacers', 34, 38),
 (135, 'Charlotte Hornets', 33, 39),
 (136, 'Chicago Bulls', 31, 41),
 (159, 'Toronto Raptors', 27, 45),
 (137, 'Cleveland Cavaliers', 22, 50),
 (153, 'Orlando Magic', 21, 51),
 (140, 'Detroit Pistons', 20, 52)]

In [91]:
params = "/games?h2h=132-134&season=2020-2021"
response = requests.get(url + params, headers=headers).json()["response"]
print(json.dumps(response, indent=4, sort_keys=True))

[
    {
        "country": {
            "code": "US",
            "flag": "https://media.api-sports.io/flags/us.svg",
            "id": 5,
            "name": "USA"
        },
        "date": "2020-12-31T00:30:00+00:00",
        "id": 104109,
        "league": {
            "id": 12,
            "logo": "https://media.api-sports.io/basketball/leagues/12.png",
            "name": "NBA",
            "season": "2020-2021",
            "type": "League"
        },
        "scores": {
            "away": {
                "over_time": null,
                "quarter_1": 41,
                "quarter_2": 27,
                "quarter_3": 36,
                "quarter_4": 37,
                "total": 141
            },
            "home": {
                "over_time": null,
                "quarter_1": 38,
                "quarter_2": 29,
                "quarter_3": 35,
                "quarter_4": 43,
                "total": 145
            }
        },
        "stage": null,
        "status"

In [93]:
ids = []
for res in response:
    ids.append(res["id"])
ids

[104109, 104124, 104326]

In [86]:
params = "/games?id=137340"
response = requests.get(url + params, headers=headers).json()["response"]
print(json.dumps(response, indent=4, sort_keys=True))

[
    {
        "country": {
            "code": "US",
            "flag": "https://media.api-sports.io/flags/us.svg",
            "id": 5,
            "name": "USA"
        },
        "date": "2021-11-03T23:30:00+00:00",
        "id": 137340,
        "league": {
            "id": 12,
            "logo": "https://media.api-sports.io/basketball/leagues/12.png",
            "name": "NBA",
            "season": "2021-2022",
            "type": "League"
        },
        "scores": {
            "away": {
                "over_time": null,
                "quarter_1": 28,
                "quarter_2": 31,
                "quarter_3": 20,
                "quarter_4": 29,
                "total": 108
            },
            "home": {
                "over_time": null,
                "quarter_1": 35,
                "quarter_2": 26,
                "quarter_3": 34,
                "quarter_4": 22,
                "total": 117
            }
        },
        "stage": null,
        "status"